In [9]:
from PIL import Image
import numpy as np
from numpy import genfromtxt
from sklearn.decomposition import PCA, TruncatedSVD
import random
from scipy.stats import multivariate_normal

#ORDEN DE REVISIÓN SUGERIDO: 
# 1) kmeans()
# 2) load()
# 3) iteration()
# 4) distance()
#EL MAIN SE ENCUENTRA AL PIE DEL ARCHIVO


def distance(a, b):
    #DISTANCIA EUCLIDIANA N DIMENSIONAL
    aux = np.sum(np.square(a - b))
    return np.sqrt(aux)


def iteration(k_puntos, puntos, dim_datos):
    #CALCULO DE DISTANCIA DE CADA DATO CON LOS K CENTROIDES DE LOS CLUSTERES Y ASIGNAMIENTO DEL CLUSTER MAS CERCANO PARA CADA DATO
    clusters = []
    for idx, punto in enumerate(puntos):
        d = 99999999999991
        ki = 0
        for idx_k, k_i in enumerate(k_puntos):
            if(d > distance(punto, k_i)):
                d = distance(punto, k_i)
                ki = idx_k
        clusters.append(ki)




    #CONTABILIZACION DE ELEMENTOS POR CADA CLUSTER Y SUMA DE FEATURES PARA CADA UNO DE ELLOS
    aux = [0] * dim_datos
    means = [np.array(aux)] * len(puntos)
    count = [0] * len(puntos)
    for i in range(0,len(puntos)):
        idx = clusters[i]
        means[idx] = np.add(puntos[i], means[idx])
        count[idx] += 1


    #CÁLCULO DE LA MEDIA PARA CADA CLUSTER Y REASIGNACIÓN DE LOS K CENTROIDES
    k_p = []
    for j in range(len(means)):
        # print(means[i])
        # print(count[i])
        # break
        if(count[j] != 0):
            #print('a')
            for i in range(len(means[j])):
                means[j][i] = means[j][i]/count[j]
            k_p.append(np.array(means[j]))
    k_puntos = k_p

    #RETORNO DE DATOS
    return k_puntos, clusters

def load(k, redux,redux_type, q):

    #PROCESAMIENTO DE DATOS
    borrarcomillas = lambda x: x.replace('"', '')
    names = np.genfromtxt('clase.csv', dtype=None, delimiter=",", skip_header=1, usecols=1, encoding='utf-8', converters={1: borrarcomillas})
    data = np.genfromtxt('dataset.csv', delimiter=",")

    #REDUCCION DE DIMENSIONALIDAD
    svd = TruncatedSVD(n_components=redux)
    pca = PCA(n_components=redux)
    datos = data
    if redux_type == 0:
        datos = svd.fit(data).transform(data)
    elif redux_type == 1:
        datos = pca.fit(data).transform(data)


    '''
    maximo = 0
    minimo = 9999999999999
    for i in range(len(datos)):
        for j in range(len(datos[i])):
            #datos[i][j] = 1.08**datos[i][j]
            if maximo < datos[i][j]:
                maximo = datos[i][j]
            if minimo > datos[i][j]:
                minimo = datos[i][j]
    '''

    #VARIABLES UTILES
    n_datos = len(datos)
    dim_datos = len(datos[0])
    
    #CARGA DE PUNTOS
    puntos = []
    k_puntos = []
    for i in range(n_datos):
        puntos.append(np.array(datos[i]))

    #OBTENCIÓN DE PROMEDIOS DE CADA FEATURE Y SU DESVIACIÓN ESTANDAR

    data_ = datos.transpose()
    sd = []
    promedios = []
    for x in data_:
        sd.append(np.std(x))
        promedios.append(np.mean(x))
    

    #GENERACION DE LOS K PUNTOS PARA LOS CLUSTERES USANDO UN RANGO DETERMINADO POR EL PROMEDIO + Q*DESVIACION ESTANDAR DONDE Q ES UN NUMERO ARBITRARIO PERO QUE TENGA SENTIDO
    for i in range(k):
        aux = []
        for i in range(dim_datos):
            aux.append(random.randint(round(promedios[i]-q*sd[i]),round(promedios[i]+q*sd[i])))
        k_puntos.append(np.array(aux))

  
    #RETORNO DE LOS K PUNTOS PARA LOS CLUSTERES, LOS PUNTOS CARGADOS, LA DIMENSION DE LOS DATOS Y LOS LABELS CORRECTOS
    return k_puntos, puntos, dim_datos, names



def kmeans(k, epoch, redux, redux_type, q):

    #CARGA DE DATOS
    k_puntos, puntos, dim_datos, names = load(k, redux, redux_type, q)

    # print('k_puntos: ', k_puntos)
    # print('puntos: ', puntos)

    
    #MECANISMO PARA IMPRIMIR PORCENTAJE ACTUAL DE LA OPERACIÓN Y SABER CUANTO APROXIMADAMENTE FALTA PARA QUE TERMINE EL ALGORITMO
    suma = 0
    if(epoch >= 100):
        per = epoch / 100
        suma = 1
    else:
        per = 100/epoch
        suma = per
        
    tiempo = 0
    count = 0


    #LLAMADA AL ALGORITMO iteration() CON ITERACIONES DE REFINAMIENTO=EPOCH E IMPRESIÓN DE PORCENTAJE ACTUAL DE EJECUCIÓN DEL ALGORITMO
    clusters = []
    for i in range(epoch):
        if tiempo >= per:
            # print(str(count)+'%')
            count += suma
            tiempo = 0
        k_puntos, clusters = iteration(k_puntos, puntos, dim_datos)
        tiempo+=suma
    # print(str(100)+'%')  
            
    
    #FORMATEO CORRECTO DEL RESULTADO PARA DISPLAY 
    results = []
    for i in range(k):
        aux = []
        for j in range(len(clusters)):
            if clusters[j] == i:
                aux.append(names[j])
        results.append(aux)
    
    #IMPRESIÓN DE RESULTADOS
    count = 0
    for i in results:
        print('Cluster:', count, '\n')
        print(i)
        count +=1

    #RETORNO DE RESULTADOS
    return clusters




#MAIN
# k = cantidad de clusteres
# epochs = cantidad de iteraciones de 'refinación'
# redux = cantidad de dimensiones finales que tendrá el dataset (pca puede quejarse si le pones mas de N, siendo N la cantidad de datos)
# redux_type 0 = svd, 1 = pca
# q = cantidad de veces que se suma y resta la desviación estandar a la media para obtener un rango de generación para las features de los centroides aleatorios. 
# rango para cada feature = [ft_mean-q*ft_sd, ft_mean+q*ft_sd]

# kmeans() retorna un arreglo arr[] en donde cada posición i representa el elemento i del dataset[] ya reducido y procesado, 
# y arr[i] contiene el número de cluster que se le ha asignado. 
# El arreglo names[i] contiene el label real del elemento dataset[i]
# la reducción de dimensionalidad se hace en la función load(), dejaré un indicador para que sea evidente

# k = 7
# epochs = 100
# redux = 10
# redux_type = 0 
# q = 2
# clusters = kmeans(k, epochs, redux, redux_type, q)

#PROBLEMAS ENCONTRADOS:

#CLUSTERS CON DOS CATEGORÍAS
#Posible respuesta: Algunas categorías de labels no aparecen en un cluster propio, puede deberse a que la naturaleza de los features de dicha categoría sean demasiado similares a otra categoría, por lo que se clusteriza en el mismo sitio o que en la reducción de dimensionalidad, se hayan acortado diferencias entre dos categorías y sus labels, lo que resulta en la primera observación.

#MAS DE UN CLUSTER PARA UNA CATEGORÍA
#Posible respuesta: Los features de los datos de una misma categoría tienen tendencia a diverger, con la reducción de dimensionalidad, en dos o más grupos diferentes o (la que creo que es) dos centroides aleatorios a1 y a2 cayeron muy cerca entre sí y se formaron dos miniclusters que representan lo mismo. NOTA: como el rango de generación de cada feature para los centroides aleatorios es media+-q*desviacion estandar, si sucede la primera observación, quiere decir que los rangos probablemente resultaron muy pequeños, por lo que se generaron puntos con features similares y habría que aumentar el q para dar mas holgura.

#CLUSTERES VACÍOS
#Posible respuesta: Se generó un centroide tan lejano que no resultó ser el más cercano para ningún dato, habría que evaluar disminuir el q para disminuir la holgura de generación y crear centroides más cercanos a los puntos.



In [14]:


def new_medias(gammas, N, n_samples, n_classes, k_clusters):
  medias = np.zeros((k_clusters, n_classes))
  for i in range(k_clusters):
    media = np.full((n_classes,), 0)
    for n in range(n_samples):
      media += gammas[n,i]*dataset[n,]
    media = media/N[i]
    medias[i,] += media
  return medias

def new_covarianzas(gammas, medias, N, n_samples, n_classes, k_clusters):
  covarianzas = np.zeros((k_clusters, n_classes, n_classes))

  for i in range(k_clusters):
    covarianza = np.zeros((n_classes, n_classes))
    for n in range(n_samples):
      columna = dataset[n,] - medias[i,]
      columna = columna.reshape(n_classes, 1)
      fila = columna.reshape(1, n_classes)
      covarianza += gammas[n,i]*(columna@fila)
    covarianzas[i,:,:] += covarianza/N[i]

  return covarianzas

def new_N(gammas, k_clusters, n_samples):
  N = np.zeros((k_clusters,))
  for i in range(k_clusters):
    val = 0
    for n in range(n_samples):
      val += gammas[n,i]
    N[i,] += val

  return N

def new_pis(N, n_samples, k_clusters):
  pis = np.zeros((k_clusters,))
  for i in range(k_clusters):
    pis[i,] += N[i]/n_samples
  return pis

def new_responsibilities(pis, medias, covarianzas, n_samples, k_clusters):
  gammas = np.zeros((n_samples, k_clusters))
  for n in range(n_samples):
    for k in range(k_clusters):
      numerador = pis[k,]*multivariate_normal.pdf(dataset[n,], mean=medias[k,], cov=covarianzas[k,:,:])
      denominador = 0
      for _k in range(k_clusters):
        denominador += pis[_k,]*multivariate_normal.pdf(dataset[n,], mean=medias[_k,], cov=covarianzas[_k,:,:])
      res = numerador/denominador
      gammas[n,k] += res
  return gammas

def log_likelihood(k_clusters, medias, covarianzas, n_samples, pis):
  prob = 0
  for n in range(n_samples):
    res = 0
    for k in range(k_clusters):
      res += pis[k,]*multivariate_normal.pdf(dataset[n,], mean=medias[k,], cov=covarianzas[k,:,:])
    res = np.log(res)
    prob += res
  return prob
    

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])